In [ ]:
import os,sys
import numpy as np
import chart_studio.plotly as py
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import torch
from lartpcdataset import lartpcDataset
%load_ext autoreload
%autoreload 2

In [ ]:
dataset = lartpcDataset(root="./data")
loader = torch.utils.data.DataLoader(
    dataset=dataset,
    batch_size=1,
    shuffle=True)

In [ ]:
LABEL_NAME = {0:"electron",
             1: "gamma",
             2:"muon",
             3:"proton",
             4:"pion"}

In [ ]:
batch = next(iter(loader))
print(batch[0].shape)
label = batch[1]
print("label: ",label," :: ",LABEL_NAME[label.item()])
xaxis = np.linspace( 0, 256, endpoint=False, num=256 )
yaxis = np.linspace( 0, 256, endpoint=False, num=256 )
imgplot = {                                                                                                                                                                  
    "type":"heatmap",
    "z":batch[0].squeeze(),
    "x":xaxis,
    "y":yaxis,
    "zmin":0.0,
    "zmax":10.0,
    "colorscale":"Jet",
}

fig0 = go.Figure(data=[imgplot])
fig0.show()